In [ ]:
import gzip
import os
import pickle
import tarfile
from pprint import pprint
from typing import Dict, List, Optional

import graphistry
import dgl
# import igraph as ig
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
# import optuna
import pandas as pd
import seaborn as sns
import torch
from dgl.data import DGLDataset
from dgl.data.utils import download as dgl_download
from dgl.data.utils import load_graphs, save_graphs
from dgl.sampling import global_uniform_negative_sampling
from sentence_transformers import SentenceTransformer
from sklearn.model_selection import train_test_split

from graphml_class.clean import clean_graph
from graphml_class.palette import CATEGORICAL_PALETTE

In [ ]:
SEED = 31337

# Graph Machine Learning by Hand

The old-school method of machine learning that was common practice before graph embeddings and Graph Neural Networks was to hand engineer features of the nodes of a network by querying the network with `networkx` or a graph database like Neo4j using Cypher or Gremlin and to assign the results of those queries to nodes.

What are some common features when hand engineering node features in graph machine learning tasks?

## Hand Engineered Features in Graph ML

* Scalable Centralities: Degree [in, out, total], Eigenvector, PageRank [a form of eigenvector centrality], Katz
* Less Scalable Centralities: Closeness, Betweenness

You can compute these methods on single edges or on multiple edges as a way of projecting a _higher-order_ network, a network composed of more complex semantics built from the base network. Graph ML worked this way until around 2015, when graph embeddings changed everything :)

Check out [networkx.algorithms.centrality](https://networkx.org/documentation/stable/reference/algorithms/centrality.html) and find some that fit different aspects of networks you'd like to include in a classifier.

## Loading our Citation Graph

In Part 1 on knowledge graph construction, we saved our graph in GEXF format. Let's load it using the same procedure we performed in Part 2. I'm going to abbreviate the comments here, look at Part 2 for more details on what I am doing.

In [ ]:
G: nx.DiGraph = nx.read_gexf(path="data/physics_labeled.gexf.gz")

In [ ]:
G.number_of_nodes(), G.number_of_edges()

In [ ]:
with open("data/citation/file_to_net.pkl", "rb") as f:
    file_to_net = pickle.load(f)

# Everything ok? Yes!
list(file_to_net.items())[0:10]

In [ ]:
with open("data/citation/net_to_file.pkl", "rb") as f:
    net_to_file = pickle.load(f)

# Everything ok here too? Yes!
list(net_to_file.items())[0:10]

In [ ]:
def convert_ids_to_int(G):
    # Create a new directed graph
    G_int = nx.DiGraph()
    
    # Create a mapping from string IDs to integer IDs
    id_mapping = {str_id: int(str_id) for str_id in G.nodes()}
    
    # Copy nodes and attributes, converting IDs to integers
    for str_id, data in G.nodes(data=True):
        int_id = id_mapping[str_id]
        G_int.add_node(int_id, **data)
    
    # Copy edges and attributes, converting IDs to integers
    for str_id1, str_id2, data in G.edges(data=True):
        int_id1, int_id2 = id_mapping[str_id1], id_mapping[str_id2]
        G_int.add_edge(int_id1, int_id2, **data)
    
    return G_int

# Convert G to use integer IDs
G_int = convert_ids_to_int(G)

In [ ]:
G_int.number_of_nodes(), G_int.number_of_edges()

In [ ]:
# Let's test or integer index now... we pickled it instead of JSONized it so it would retain its integer keys and values!
test_id = file_to_net[9711194]
G_int.nodes[test_id]

### `G_int` --> `G`

Now we can assign our new integer graph back to `G` and use it below :)

In [ ]:
G = G_int

## Feature Engineering for Node Classification

We are going to load our citation graph and then combine a simple encoding of the text features with manually computed features to perform node classification. We will see how our network metrics describing topology affect performance. Try not to always jump to using a network embedding, this is a useful exercise as a baseline :)

Our job is to prepare a network features vector `features` that we can append to our other features before we train a classifier to perform link prediction.

### Historical Review

Remember these slides? Think on them again :) We are going to implement a couple of these to drive our first Journal classifier :) This slide shows numerous methods used in link prediction.

<center><img src="images/Feature-Engineering-for-Link-Prediction.jpg" width="1000px" /></center>
<center><i>Feature engineering for link predicton can be complicated due to the need to compute them for pairs of nodes.</i></center>

<br />

<center><img src="images/JanusGraph-Titan-Feature-Engineering-in-2014.png" width="1000px" /></center>
<center><i>Feature engineering circa 2014 on a business graph representing relationships between companies in a property graph using <a href="https://titan.thinkaurelius.com/">Titan</a> via <a href="https://tinkerpop.apache.org/gremlin.html">Tinkerpop / Gremlin</a>. Titan became <a href="https://janusgraph.org/">JanusGraph</a>.</i></center>

<br />

### Store Features by Column in `features`

We will compute features for each node or pairs of nodes and then store them in the `features` `np.ndarray` by appending one column at a time. One row is a node. This is described in [NumPy: How to add an extra column to a NumPy array](https://www.w3resource.com/python-exercises/numpy/python-numpy-exercise-86.php) as looking like:

<center><img src="images/append-column-to-matrix.png" width="500px" /></center>
<center>Image Source: <a href="https://www.w3resource.com/python-exercises/numpy/python-numpy-exercise-86.php">w3resource: NumPy: How to add an extra column to a NumPy array</a></center>

#### Re-Running Feature Appends Will Make Duplicate Feature Columns

Warning: If you run the cells below that do `np.append(features, my_feat_ary, axis=1)` more than once... you will get a duplicate features column.

In [ ]:
# I had to look this up, so here you are :)
import numpy as np
x = np.array([[10,20,30], [40,50,60]])
y = np.array([[100], [200]])
x, y, np.append(x, y, axis=1)

### Initialize `features` `nd.array`

You can re-run this to recalculate the features from scratch.

In [ ]:
# Create an inner list for each node ID
inner_lists = [[] for x in range(G.number_of_nodes())]

# The shape is 27,770 nodes long with zero feature columns
features = np.array(inner_lists)
features, features.shape

### `networkx` Centrality Metrics

For our first features, we will use `networkx` to compute some centrality metrics. I'm going to limit our calculations to two centralities you can use in practice on networks of most any size:

* Degree Centrality (in degree, out degree, degree [both]) - this is simple and shows how well connected a node is
* Eigenvector Centrality - measures the influence of a node in a network, where a node is considered influential if it is connected to other influential nodes.

Check out this list of [networkx.centrality](https://networkx.org/documentation/stable/reference/algorithms/centrality.html) metrics for feature ideas.

#### Display Metrics using `pd.Series`

If you want to see a metric computer by `nx.my_function()` you can wrap the `Dict` it returns [which will ALL print in a notebok] in a `pd.Series` and it will use the node ID as the index and display the feature. It is a good idea to lay eyes on things you compute - when possible - as often something goes wrong on a first attempt :)

#### Degree Centrality

Degree centrality is a **local centrality** metric. It measure influence in the nearby network.

NetworkX measures degree centrality as a relative value for all nodes in a network. This can complicate inference but keeps values within an interpretable range.

In [ ]:
degree = nx.degree_centrality(G)

# See - takes just 11 lines. The Dict printed over 27K rows on my screen. Try it!
pd.Series(degree)

In [ ]:
in_degree = nx.in_degree_centrality(G)
pd.Series(in_degree)

In [ ]:
out_degree = nx.out_degree_centrality(G)
pd.Series(out_degree)

#### Eigenvector Centrality

Eigenvector centrality is a **global centrality** metric. It measures influence within the entire network. _We always want a local and global centrality metric in our feature set_. To understand eigenvectors, check out this amazing simulation: [Eigenvectors and Eigenvalues](https://setosa.io/ev/eigenvectors-and-eigenvalues/)

In [ ]:
# Eigenvector centrality is problematic in disconnected graphs, so take the largest component...
components = nx.connected_components(G.to_undirected())
largest_component = max(components, key=len)
G_big_component = G.subgraph(largest_component).copy()

print(f"Largest comonent size: {len(G_big_component):,}")

In [ ]:
# Visualize the eigenvectors... see 
eigenvector = nx.eigenvector_centrality_numpy(G_big_component.to_undirected())
pd.Series(eigenvector)

In [ ]:
# ... and make it undirected, to ensure it is connected
eigenvectors = nx.eigenvector_centrality_numpy(
    G_big_component.to_undirected()
)

for node, score in eigenvectors.items():
    G_big_component.nodes[node]["eigenvector"] = score
    G_big_component.nodes[node]["eigenvector_log10"] = np.log10(score + 0.000001)  # can't be zero for log

In [ ]:
# Initialize eigenvector_centrality property as 0 for all nodes
nx.set_node_attributes(G, 0.0, "eigenvector")
nx.set_node_attributes(G, 0.0, "eigenvector_log10")

# Assign the eigenvector centrality values from subgraph to original graph
nx.set_node_attributes(G, {node: eigenvectors[node] for node in G_big_component}, "eigenvector")
nx.set_node_attributes(G, {node: np.log10(G.nodes[node]['eigenvector'] + 1e-10) for node in G.nodes()}, "eigenvector_log10")

In [ ]:
eigenvector_dict = nx.get_node_attributes(G, 'eigenvector')

In [ ]:
# Get an array for our features array
eigenvector_array = np.array([G.nodes[node]["eigenvector"] for node in G.nodes()])
pd.Series(eigenvector_array)

### Add to `features` `np.array`

That gives us four features so far that indicate a node's prominence in the network. Let's combine them to produce a four feature long feature vector.

In [ ]:
np.array(list(degree.values())[0:10]), np.array(list(degree.values())[0:10]).reshape(-1, 1)

In [ ]:
# Prepare and append the degree features for addition to `features`
for d in [degree, in_degree, out_degree, eigenvector_dict]:
    d_ary = np.array(list(d.values())).reshape(-1, 1)
    features = np.append(features, d_ary, axis=1)

features, features.shape

### Community Detection with Louvain Modularity

What about other kinds of features? One important category is the local group a node belongs to after performing an operation called 

One thing we didn't cover in Part 2 - Network Science, was community detection. What community a node belongs to is an important feature. We're going to cluster our network into communities and visualize them in Graphistry. Then we're going to add the cluster IDs as features in our `features` `np.array`.

While there are many network clustering algorithms, the most common method of community detection is [Louvain Modularity](https://en.wikipedia.org/wiki/Louvain_method). We're going to use [nx.community.louvain_communities](https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.community.louvain.louvain_communities.html#networkx.algorithms.community.louvain.louvain_communities) because it is efficient enough to work on networks up to moderate size using one computer. You can usually find a Louvain Modularity implementation for whatever platform you use to process even large networks - big data.

> The method is a greedy optimization method that appears to run in time `O(n * log(n))`.
>
> ...
> 
> In the Louvain Method of community detection, first small communities are found by optimizing modularity locally on all nodes, then each small community is grouped into one node and the first step is repeated.

-- Wikipedia, [Louvain method](https://en.wikipedia.org/wiki/Louvain_method)

In [ ]:
clusters = nx.community.louvain_communities(G, seed=SEED)
len(clusters)

#### Evaluating Modularity

I am curious how many clusters there are... lets plot their size in Seaborn using log scale. You can see there is a power law distribtion in cluster size.

In [ ]:
# Get connected components and their sizes
cluster_size = [len(c) for c in clusters]

# Use seaborn to create the histogram
sns.histplot(cluster_size, kde=True, bins=20, log_scale=True)
plt.title("Histogram of Cluster Sizes")
plt.xlabel("Cluster Size")
plt.ylabel("Count")
plt.show()

In [ ]:
pprint(sorted(cluster_size, reverse=True)[0:50], indent=4, compact=True)

### ChatGPT and Me

I needed to do consult a reference to interpret what this might mean for a citation graph... I knew it had a power law distribution with a long tail. I fed the following prompt to ChatGPT-4 Browser:

#### **CHATGPT PROMPT**

I have a citation graph of physics papers from arXiv, the high energy physics citation network from Stanford SNAP. I used NetworkX to compute Louvain Modularity and measured cluster sizes. Then I plotted them in seaborn on a log scale.

The first 50 cluster sizes, sorted in descending order are:

```
[   2481, 2144, 1779, 1731, 1727, 1596, 1594, 1551, 1447, 1412, 1332, 1255,
    1108, 919, 844, 835, 817, 674, 540, 495, 316, 314, 170, 157, 113, 11, 10,
    10, 8, 8, 7, 7, 6, 6, 6, 5, 5, 5, 5, 5, 5, 4, 4, 4, 4, 4, 4, 4, 4, 4]
```

Please interpret this histogram of cluster sizes and tell me what it means for a citation graph.

#### Comment

The output checks out... ChatGPT-4 can interpret a histogram in a specific domain fairly well. Check it out. I'm trying to demonstrate how to incorporate AI into your workflows... I did ask it to rewrite this in fewer words, but I'm printing the entire output below.

#### **CHATGPT RESPONSE**

The cluster sizes you've provided, derived from applying the Louvain Modularity algorithm on a citation graph of physics papers, represent the sizes of communities of papers that cite each other. The sizes are listed in descending order, indicating a few large clusters followed by many smaller ones. This pattern is typical in real-world networks and is often referred to as a scale-free or heavy-tailed distribution. Here are a few interpretations and insights based on this distribution:

* Community Structure - There are a few large communities of papers (e.g., the largest has 1909 papers) which likely represent major topics or fields within high energy physics. These large clusters suggest that there are common themes that have garnered significant attention and citation in the community.
* Long Tail - As we move down the list, the cluster sizes decrease, forming a long tail of smaller communities. This indicates a wide variety of specialized or niche topics within the field that have smaller groups of papers citing each other.
* Power-Law Distribution - The distribution of cluster sizes resembles a power-law distribution, which is common in many natural and social phenomena. In a power-law distribution, a few items (in this case, topics or fields) are exceptionally common, but the vast majority are rare.
* Interdisciplinary or Emerging Areas - The smaller clusters may represent interdisciplinary or emerging areas of research that have not yet amassed a large number of citations.
* Potential for Further Exploration - Understanding the thematic content of the large clusters could provide insights into the major areas of focus in high energy physics, while exploring the smaller clusters might reveal emerging or under-explored areas of research.
* Rich-Get-Richer Phenomenon - The distribution may also reflect a "rich-get-richer" phenomenon where well-cited papers continue to accumulate more citations over time, reinforcing the prominence of certain topics.
* Validation of Louvain Modularity - The distribution of cluster sizes could also serve as a validation of the Louvain Modularity algorithm's effectiveness in detecting community structure within the citation network.

### Featurizing Communities

To featurize our communities, we need to assign numeric values to the clusters and append a column for cluster ID to `features`. Note, we could alternatively [one-hot-encode](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html) the features to give them one feature column per cluster. This would make the features sparse, which means lesss signal - something neural networks don't like but that another algorithm might not mind. We could use the [hashing trick](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.FeatureHasher.html) - a method of feature engineering somewhere in between one-hot-encoding and neural embeddings. We are instead going to _label encode_ them with [sklearn.preprocessing.LabelEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html).

In [ ]:
# for cluster in clusters:
pprint(clusters[0:3], indent=4, compact=True)

In [ ]:
# Iterate through clusters, assigning cluster IDs, then map them to node IDs
node_clusters: Dict[int, int] = {}
for cluster_id, cluster in zip(range(len(clusters)), clusters):
    for node_id in cluster:
        node_clusters[node_id] = cluster_id

node_series = pd.Series(node_clusters).sort_index()
node_series

### Append Clusters to `features`

Note, the clusters number from approximately 1-1,500. This dwarfs the previously computed, normalized centrality scores. We will be using scikit-learn's [`StandardScaler`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) to address this.

Now we've got 5 topological features. We are going to add one more and then do a simple feature extraction of our Titles and Abstracts that might normally go with a first pass at a node classifier.

In [ ]:
# Prepare and append the degree features for addition to `features`
cluster_ary = np.array(node_series).reshape(-1, 1)
features.shape, cluster_ary.shape

In [ ]:
features = np.append(features, d_ary, axis=1)
features, features.shape

## Role Discovery via RolX and ReFeX

RolX role discovery is one of my favorite algorithms for networks :) You may recall from the slides how I used it to build a visually appealing, intrepretable map of the big data market. We're going to compute RolX roles to use as feature for ML tasks.

### RolX

You can learn more about RolX in the paper [RolX: Structural Role Extraction & Mining in Large Graphs](https://web.eecs.umich.edu/~dkoutra/papers/12-kdd-recursiverole.pdf).

<center><img src="images/RolX-Roles-vs-Communities.png" width="550px" /></center>
<center><i>You can see the difference between structural roles and communities. A related concept called _network motifs_ or structural patterns in networks can determine the nodes roles play in their neighborhoods.</i></center>

<br /><br />

<center><img src="images/Graph-ML-Class-Slide-for-Rolx-Roles-for-Mapping-Big-Data.png" width="1000px" /></center>
<center><i>An example of RolX in action to filter nodes with roles I didn't care about from a visualization to tell the story of interest.</i></center>

### ReFeX via `graphrole`

ReFex is a role discovery algorithm similar to RolX. It was defined in [It’s Who You Know: Graph Mining Using Recursive Structural Features](https://www.andrew.cmu.edu/user/lakoglu/pubs/ReFeX.pdf).

GraphRole draws from Rolx and ReFeX to do role discovery. Note that the `graphrole` implementation of RolX is a little slow... even on this network. The reality is that for large networks, you need the support of a data engineer to implement algorithms. It isn't feasible to teach a lot of large network computing skills in a five day, four hour course. RolX itself was designed to support large networks, and there are other implementations available.

Therefore I have pickled the results and made them available for you to load. **Feel free to uncomment the next block of code out and run it if you like, otherwise the RolX features are computed and stored in Github.** They are loaded below.

In [ ]:
from graphrole import RecursiveFeatureExtractor, RoleExtractor


# Extract features
feature_extractor = RecursiveFeatureExtractor(G)
rolx_features = feature_extractor.extract_features()

# Calculate roles
role_extractor = RoleExtractor(n_roles=None)
role_extractor.extract_role_factors(rolx_features)

# Can also use role percentages
role_extractor.role_percentage

In [ ]:
node_roles = role_extractor.roles
node_role_s = pd.Series(node_roles)
node_role_s

In [ ]:
def get_roles() -> pd.Series:
    """Compute the roles unless they are already computed."""
    # Extract features
    feature_extractor = RecursiveFeatureExtractor(G)
    rolx_features = feature_extractor.extract_features()
    
    # Calculate roles
    role_extractor = RoleExtractor(n_roles=None)
    role_extractor.extract_role_factors(rolx_features)

    # return pd.Series(role_extractor.role_percentages)
    return pd.Series(role_extractor.roles)

In [ ]:
# Save the feature_extractor and role_extractor via pickle
node_role_path = "data/citation/node_role_s.pkl"
node_role_s: Optional[pd.Series] = None
if os.path.exists(node_role_path):
    with open(node_role_path, "rb") as f:
        node_role_s = pickle.load(f)
else:
    node_role_s = get_roles()
    with open(node_role_path, "wb") as f:
        pickle.dump(node_role_s, f)

node_roles = node_role_s.to_dict()

In [ ]:
node_role_s, node_role_s.shape

#### Label Encoding Roles with [sklearn.preprocessing.LabelEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html)

We need numeric role numbers, not strings, so we can add the `node_roles` to our `features` and start classifying. This is what scikit-learn's `LabelEncoder` does.

In [ ]:
from sklearn.preprocessing import LabelEncoder


le = LabelEncoder()
flat_role_feature = le.fit_transform(node_role_s)
flat_role_feature, flat_role_feature.shape

In [ ]:
role_feature = flat_role_feature.reshape([-1,1])
role_feature, role_feature.shape

#### Add `role_feature` to `features`

In [ ]:
features = np.append(features, role_feature, axis=1)
features, features.shape

## Visualizing Communities and Roles in Graphistry

We're going to use `graphistry` to visualize the communities and roles we determined to see if we can interpret them the way ChatGPT did - or in some other way :) This sort of belongs in Part 2 - Network Science, but we saved it here to use as a feature for node classification.

In [ ]:
nx.set_node_attributes(G, values=node_roles, name="rolx_role")

In [ ]:
nx.set_node_attributes(G, values=node_clusters, name="community")

In [ ]:
G.nodes[1]

In [ ]:
from littleballoffur import PageRankBasedSampler


G_orig = G.copy()
sampler = PageRankBasedSampler(10000)
G_sample = nx.convert_node_labels_to_integers(
    sampler.sample(G_orig.to_undirected())
)

G_sample.number_of_nodes(), G_sample.number_of_edges()

In [ ]:
# Environment variable setup
GRAPHISTRY_USERNAME = os.getenv("GRAPHISTRY_USERNAME")
GRAPHISTRY_PASSWORD = os.getenv("GRAPHISTRY_PASSWORD")

In [ ]:
graphistry.register(
    api=3,
    username=GRAPHISTRY_USERNAME,
    password=GRAPHISTRY_PASSWORD,
)

In [ ]:
# Configuration for Graphistry
FAVICON_URL = "https://graphlet.ai/assets/icons/favicon.ico"
LOGO = {"url": "https://graphlet.ai/assets/Branding/Graphlet%20AI.svg", "dimensions": {"maxWidth": 60, "maxHeight": 60}}

In [ ]:
# Parameters for GRAPHISTRY
GRAPHISTRY_PARAMS = {
    "play": 500,
    "pointOpacity": 0.7,
    "edgeOpacity": 0.3,
    "edgeCurvature": 0.3,
    "showArrows": True,
    "gravity": 0.15,
    "showPointsOfInterestLabel": False,
    "labels": {
        "shortenLabels": False,
    },
    "scalingRatio": 0.5,
}

In [ ]:
g = (
    graphistry.bind(
        source="src",
        destination="dst",
        node="nodeid",
        point_title="Title",
        point_label="Title",
    )
    .addStyle(
        page={
            "title": "Community Detection Plot",
            "favicon": FAVICON_URL
        },
        logo=LOGO,
    )
    .settings(
        url_params=GRAPHISTRY_PARAMS,
        height=800,
    )
)

In [ ]:
g.plot(G_sample)

## Classifying Nodes into Journals

We are going to train a classifier and then classify the nodes without journal entries into the most common journals in the field of high energy physics to see where they most likely belong. We are going to use a simple algorithm to do this, then get more sophisticated in the next sections on _graph embeddings_ and _graph neural networks (GNNs)_.

In [ ]:
journal_labels = np.array([
    x[1]["Journal-ref-DBSCAN"] for x in list(G.nodes(data=True))
])
journal_labels, journal_labels.shape

In [ ]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(
    features,
    journal_labels,
    test_size=0.25,
    random_state=SEED
)

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score


def score_classifier(y_true, y_pred):
    """Run and return a pd.DataFrame of various classifier test metrics."""
    return {
        "Metric": ["Accuracy", "Precision", "Recall", "F1"],
        "Score": [
            accuracy_score(y_true, y_pred),
            precision_score(y_true, y_pred, average="weighted"),
            recall_score(y_true, y_pred, average="weighted"),
            f1_score(y_true, y_pred, average="weighted"),
        ],
    }

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler


scaler = StandardScaler()
clf = make_pipeline(StandardScaler(), LogisticRegression())

In [ ]:
model = clf.fit(X_train, y_train)
model

In [ ]:
y_pred = model.predict(X_test)

pd.DataFrame(score_classifier(y_test, y_pred))

## Exercise: Try to improve the perormance of our model using more `networkx` metrics

Take a look at the [long list of algorithms provided by networkx]() including its own [node classification methods](https://networkx.org/documentation/stable/reference/algorithms/node_classification.html). Compute several new features and add them to the `features` `np.array` as above, before re-scoring our classifier.

In [ ]:
# from networkx.algorithms import node_classification
#
# harmonic_class = node_classification.harmonic_function(G.to_undirected(), label_name="Journal-ref-DBSCAN")
# harmonic_class_ary = np.array(harmonic_class)
# harmonic_class_ary, harmonic_class_ary.shape

In [ ]:
# features = np.append(
#     features,
#     harmonic_class_ary.reshape(-1, 1),
#     axis=1
# )
# features, features.shape

In [ ]:
# consist_class = node_classification.local_and_global_consistency(G.to_undirected(), label_name="Journal-ref-DBSCAN")
# consist_class_ary = np.array(consist_class)
# consist_class_ary, consist_class_ary.shape

In [ ]:
# features = np.append(
#     features,
#     consist_class_ary.reshape(-1, 1),
#     axis=1
# )
# features, features.shape

In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(
#     features,
#     journal_labels,
#     test_size=0.25,
#     random_state=SEED
# )

# model = clf.fit(X_train, y_train)
# y_pred = model.predict(X_test)

# pd.DataFrame(score_classifier(y_test, y_pred))

## Exercise: Improve the model to improve performance

SVC may not be the best model for this type of work - can you improve the classification using a different type of model? Look at [scikit-learn's supervised learning catalogue](https://scikit-learn.org/stable/supervised_learning.html) for some additional models to try.

### Implement a model in XGBoost

XGBoost is a popular library for building high performance classifiers. Let's implement a classifier using an [XGBClassifier](https://xgboost.readthedocs.io/en/latest/python/python_api.html#xgboost.XGBClassifier).

In [ ]:
# from xgboost import XGBClassifier


# # Get number of unique classes
# n_classes = np.unique(journal_labels).size

# # Replace LogisticRegression with XGBoost
# xgb_clf = XGBClassifier(
#     objective='multi:softmax',
#     num_class=n_classes,  # Add this parameter
#     random_state=SEED,
#     n_estimators=100,
#     learning_rate=0.1
# )

# # Fit the model - note that XGBoost doesn't require scaling
# model = xgb_clf.fit(X_train, y_train)

# # Make predictions
# y_pred = model.predict(X_test)

# # Score the model
# pd.DataFrame(score_classifier(y_test, y_pred))

## Topology and Text Features

Now that we've improved the topological features of our classifier, let's try adding the abstract paraphrase embedding to see how our model performs.

In [ ]:
from sklearn.decomposition import PCA
embedded_abstracts = np.load("data/embedded_abstracts.npy")

pca = PCA(n_components=32)
pca_abstracts = pca.fit_transform(embedded_abstracts)
pca_abstracts, pca_abstracts.shape

In [ ]:
super_features = np.append(features, pca_abstracts, axis=1)
super_features, super_features.shape

In [ ]:
from sklearn.ensemble import RandomForestClassifier


scaler = StandardScaler()
clf = make_pipeline(StandardScaler(), RandomForestClassifier())

X_train, X_test, y_train, y_test = train_test_split(
    super_features,
    journal_labels,
    test_size=0.25,
    random_state=SEED
)

model = clf.fit(X_train, y_train)
y_pred = model.predict(X_test)
pd.DataFrame(score_classifier(y_test, y_pred))

### Graphs > Text

Isn't it interesting that a sentence embedding of the entire abstract can tell a classifier for academic journal less than a few hand-computed network metrics?

## Excrcise: Graph + Text Feature Engineering

See if you can feature engineer the graph and text features together along with the model selection or model parameters to produce improved performance in our node classifier!

# Graph Machine Learning with Graph Embeddings

[DeepWalk](https://arxiv.org/abs/1403.6652), Perozzi et al., 2014, was a revolution in graph machine learning. Along with [node2vec](https://snap.stanford.edu/node2vec/), Grover, A.; Leskovec, J, 2016, which came with code on Github [[aditya-grover/node2vec](https://github.com/aditya-grover/node2vec)], it removed the need to spend as much time doing feature engineering by hand. Although initially these embeddings worked on simple graphs with one type of edge and ignore node and edge properties... by efficiently encoding topology around a node, they autoomated much of the work involved in tasks like node classification/labeling and link prediction.

## Sampling Our Network

We will need to sample our network to compute some of these graph embeddings in a timeline manner. Remember our work from Part 3 - we will need it.

In [ ]:
from littleballoffur import PageRankBasedSampler


G_orig = G.copy()
sampler = PageRankBasedSampler(5000)
G_sample = nx.convert_node_labels_to_integers(
    sampler.sample(G_orig.to_undirected())
)

G_sample.number_of_nodes(), G_sample.number_of_edges()

In [ ]:
journal_labels = np.array([x[1]["Journal-ref-DBSCAN"] for x in list(G_sample.nodes(data=True))])
journal_labels, journal_labels.shape

In [ ]:
# DeepWalk was first, but it hardly state of the art. Let's see what it can do...
from karateclub import DeepWalk


d = DeepWalk()
d.fit(G_sample)
deepwalk_features = d.get_embedding()

deepwalk_features, deepwalk_features.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    deepwalk_features,
    journal_labels,
    test_size=0.25,
    random_state=SEED
)

clf = RandomForestClassifier()
model = clf.fit(X_train, y_train)
y_pred = model.predict(X_test)

pd.DataFrame(score_classifier(y_test, y_pred))

### Node2Vec

After DeepWalk came Node2Vec. Let's try it!

In [ ]:
from karateclub import Node2Vec


# Default: walk_number=10, walk_length=80
d = Node2Vec(walk_number=8, walk_length=16)
d.fit(G_sample)
node2vec_features = d.get_embedding()

node2vec_features, node2vec_features.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    node2vec_features,
    journal_labels,
    test_size=0.25,
    random_state=SEED
)

clf = RandomForestClassifier()
model = clf.fit(X_train, y_train)
y_pred = model.predict(X_test)

pd.DataFrame(score_classifier(y_test, y_pred))

### LaplacianEigenmaps

LaplacianEigenmaps is a newer embedding, let's see how it performs...

In [ ]:
from karateclub import LaplacianEigenmaps


d = LaplacianEigenmaps()
d.fit(G_sample)
laplacian_features = d.get_embedding()

laplacian_features, laplacian_features.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    laplacian_features,
    journal_labels,
    test_size=0.25,
    random_state=SEED
)

clf = RandomForestClassifier()
model = clf.fit(X_train, y_train)
y_pred = model.predict(X_test)

pd.DataFrame(score_classifier(y_test, y_pred))

Before we are too hard on our classifiers, keep in mind that a text model did just as poorly as many of these. It seems likely that articles can appear in multiple physics journals and that there is some chance as to which one a given paper is accepted in.

### Exercise: Compare Network Clusters with Journal Labels

One way to check this is to go back to our visualization of communities above and compare them to our labels. If they don't match up, the papers may not be differentiable on that basis.

### Exercise: Titles, Not Abstracts

What about the abstracts? Maybe they are the problem. Reduce them to 2-dimensions as we did the Journal embeddings and cluster them to see how differentiable they are. Then make an informed decision about how tenable a task our journal classification is.

## Link Prediction

Our next topic is link prediction, a common task in graph machine learning. We will use it to predict whether one paper cites another - this could be useful for missing citations or for predicting future citations.

In [ ]:
from littleballoffur import RandomNodeEdgeSampler


# Link prediction features are expensive to compute - sample edges from our graph
edge_sampler = RandomNodeEdgeSampler(1000) 
sampled_edges = list(edge_sampler.sample(G.to_undirected()).edges())
negative_sampled_edges = list(
    map(
        tuple, np.random.choice(G.nodes, (1000,2))
    )
)
all_edges = sampled_edges + negative_sampled_edges

# Set positive and negative labels for our edges
pair_labels = [[1] * len(sampled_edges) + [0] * len(negative_sampled_edges)]
pair_labels_ary = np.array(pair_labels).reshape([-1])

# Compute some features
jaccards = list(nx.jaccard_coefficient(G.to_undirected(), all_edges))
ra = list(nx.resource_allocation_index(G.to_undirected(), all_edges))
aai = list(nx.adamic_adar_index(G.to_undirected(), all_edges))
pa = list(nx.preferential_attachment(G.to_undirected(), all_edges))
csh = list(nx.cn_soundarajan_hopcroft(G.to_undirected(), all_edges))
rish = list(nx.ra_index_soundarajan_hopcroft(G.to_undirected(), all_edges))

In [ ]:
pair_features = []
for i, pair in enumerate(all_edges):
    featureset = []
    for metric in jaccards[i], ra[i], aai[i], pa[i], csh[i], rish[i]:
        featureset.append(metric[-1])
    pair_features.append(featureset)

In [ ]:
pair_features_ary = np.array(pair_features)
pair_features_ary, pair_features_ary.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    pair_features_ary,
    pair_labels_ary,
    test_size=0.25,
    random_state=SEED
)

clf = RandomForestClassifier()

model = clf.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)
score_classifier(y_test, y_pred)

## Exercise: Link Prediction with Node Features

Manual link prediction can use the same features that node classification can, if you provide features for both nodes. Improve the above link prediction by adding the previously computed node features for each node in each pair of nodes in the training and test data.

## Exercise: Link Prediction with Embeddings

Pairs of embeddings can do a better job of link prediction. Append the embeddings previously computed and see if you can improve the performance of our link prediction.

In [ ]:
# DeepWalk was first, but it hardly state of the art. Let's see what it can do...
from karateclub import DeepWalk


d = DeepWalk()
d.fit(G)
deepwalk_features = d.get_embedding()

deepwalk_features, deepwalk_features.shape

In [ ]:
pair_embeddings = []

for edge_pair in all_edges:
    feature_row = np.append(
        deepwalk_features[edge_pair[0]],
        deepwalk_features[edge_pair[1]]
    )
    pair_embeddings.append(feature_row)

pair_features_ary = np.stack(pair_embeddings)
pair_features_ary, pair_features_ary.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    pair_features_ary,
    pair_labels_ary,
    test_size=0.25,
    random_state=SEED,
    shuffle=True,
)

clf = RandomForestClassifier()

model = clf.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)
score_classifier(y_test, y_pred)

# Graph Machine Learning with Graph Neural Networks (GNNs)

Having explored network science, we are about to dive into Graph Neural Networks (GNNs). The best introduction to GNNs is a long blog post by []() entitled [A Gentle Introduction to Graph Neural Networks](https://distill.pub/2021/gnn-intro/) which the authors have _generously_ licensed under the Creative Commons. This lets me utilize their work to explain how GNNs work while providing source code along with it to bring your theoretical understanding to a practical one.

## Citation: A Gentle Introduction to Graph Neural Networks

Parts of the content in Part 4 of this course are based upon: `Sanchez-Lengeling, et al., "A Gentle Introduction to Graph Neural Networks", Distill, 2021.` This content is cited inline. Students are encouraged to read this blog post before or after class, and to reference it if they become confused about concepts in their data science and machine learning practice. 

The full list of authors is:

* [Benjamin Sanchez-Lengeling](https://research.google/people/106640/)
* [Emily Reif](https://research.google/people/106150/)
* [Adam Pearce](https://research.google/people/AdamPearce/)
* [Alexander B. Wiltschko](https://www.linkedin.com/in/alex-wiltschko-0a7b7537/)

During the course you will have access to the instructor, who understands GNNs and can elaborate further and answer any questions you may have :)

## Why is there so much talk about Graph Neural Networks?

Knowledge graphs are at the peak of the Gartner hype cycle and graph neural networks (GNNs) are soon to be high on the ramp because they tap and unlock the potential of enterprise knowledge graphs. Data lakes put data in one place, knowledge graphs link datasets together and graph neural networks automate business processes using data from across an enterprise. 



Most graph databases are fast becoming cloud-based GNN platforms:

* Neo4j → [Neo4j Graph Data Science](https://neo4j.com/product/graph-data-science/)
* TigerGraph → [Machine Learning Workbench](https://www.tigergraph.com/ml-workbench/)
* ArangoDB → [ArrangoGraphML](https://www.arangodb.com/arangodb-for-machine-learning/)
* Kumo → [SQL query the future](https://kumo.ai/)


# Graph Neural Networks (GNNs) with DGL (Deep Graph Library)

[DGL or Deep Graph Library](https://dgl.ai) is the simplest way to get started with graph machine learning using graph neural networks (GNNs).

First we will cover a few common operations with each major task type we covered in the lecture: node-level, edge-level, subgraph-level and graph-level.

## Node-Level Tasks: Classification

Node-level tasks usually involve property prediction - classifying nodes into categories or regressing one of their numeric properties. We'll cover both.

As in the network science section of this course, we will start with a Text Attributed Graph (TAG) called a Citation Graph. We are going to use the [CORA dataset](https://relational.fit.cvut.cz/dataset/CORA), [described by Papers with Code](https://paperswithcode.com/dataset/cora) as:

> Introduced by Andrew McCallum et al. in [Automating the Construction of Internet Portals with Machine Learning](https://doi.org/10.1023/A:1009953814988)
>
> The Cora dataset consists of 2708 scientific publications classified into one of seven classes. The citation network consists of 5429 links. Each publication in the dataset is described by a 0/1-valued word vector indicating the absence/presence of the corresponding word from the dictionary. The dictionary consists of 1433 unique words.

### CORA Node Features: Bag of Words

Note... the features for this network are a [Bag of Words](https://en.wikipedia.org/wiki/Bag-of-words_model) model: simple and _sparse_ rather than modern text representations which are _dense_, distributed representations in the form of language models or [embeddings](https://cloud.google.com/blog/topics/developers-practitioners/meet-ais-multitool-vector-embeddings). Each node has a row in the feature matrix and each of 1,433 unique words get a column with the word count. Before [Word2Vec](https://arxiv.org/abs/1301.3781) introduced text embeddings in 2013, the features for NLP problems were mostly 0s, with a few non-zero values.

<center><img src="images/sparse_vs_dense_vectors.webp" width="800px" alt="Bag-of-Words (BoW) sparse vectors used in traditional NLP versus dense, embedded vector representations used in modern deep learning NLP" /></center>

The [curse of dimensionality](https://en.wikipedia.org/wiki/Curse_of_dimensionality) prevented NLP applications from realizing their modern capabilities - the more words that were added, the more dimensions the features data got and the more dimensions you add to a _sparse_ feature vector... the more all the values of that vector start to approximate the same value. They stretch out over many dimensions and look the same.

Embeddings like Word2Vec related _sparse representations_ of words to the text around them by storing a middle layer of a neural network, creating _dense representations_.

<center><img src="images/from_sparse_to_dense.webp" width="700px" alt="The Word2Vec's Skipgram architecture maps sparse to dense vectors via a shallow embedding technique" /></center>

These are very useful because the dimensions of the feature vector correspond to particular semantics, and because you can compare two dense vectors and get a sense of how similar the objects the represent are. This is very useful for information retreival applications like search and clustering.

<center><img src="images/king_minus_man_plus_woman.webp" width="800px" alt="Given the dense embedding vector for the word 'king', if we subtract the vector for 'man' and add 'woman', we arrive at a vector very close to 'queen'." /></center>

<br />

We could use a language model or large language model (LLM) to embed the features or the original text and get better performance from our GCN. However, it is good to start simple and worry about feature engineering lately... you can spend an endless amount of time over optimizing a task nobody cares about. Make sure they want the prototype before you engineer incredible performance. A Bag of Words representation is a fine start.

### CORA Classifier: Graph Convolutional Network

We are going to use a neural network architecture that may be familiar to you: a convolutional neural network. The type we will employ is called a Graph Convolutional Network (GCN). Message passing occurs between nodes and the series of input messages to a node are summarized by the layers of a GCN after each round of message passing.

<center><img src="images/gcn-decagon-overview.png" width="1000px" alt="Graph Neural Networks for Multirelational Link Prediction" /><a href="https://snap.stanford.edu/decagon/">Graph Neural Networks for Multirelational Link Prediction, Zitnik et al., 2018</a></center>


There is often a big of tinkering required to make GNNs run, so even for this simple problem in DGL, we must specify our GNN architectre. It is simple enough. Let's see how it looks...

Note: Figures Sources: [Dense Vectors: Capturing Meaning with Code](https://towardsdatascience.com/dense-vectors-capturing-meaning-with-code-88fc18bd94b9) by [James Briggs](https://jamescalam.medium.com/), [Graph Neural Networks for Multirelational Link Prediction, Zitnik et al., 2018](https://snap.stanford.edu/decagon/)

### Building a GCN in DGL

Let's build, train and evaluate our first GNN: a graph convoltional network for classifying CORA articles into categories.

Note: Source for this section is the [Blitz Tutorial, Node Classification with DGL](https://docs.dgl.ai/tutorials/blitz/1_introduction.html#sphx-glr-tutorials-blitz-1-introduction-py).

In [ ]:
import os

# DGL can also use Tensorflow or MXNet
os.environ["DGLBACKEND"] = "pytorch"
import dgl
import dgl.data
import torch
import torch.nn as nn
import torch.nn.functional as F

For now we will use a pre-loaded dataset. It contains the standard CORA bag-of-word (BoW) featres. Later we will construct our own graphs to perform feature engineering on them to do more sophisticated work.

In [ ]:
dataset = dgl.data.CoraGraphDataset()

print(f"Number of categories: {dataset.num_classes}")

In [ ]:
# There can be more than one graph, this dataset has just one
g = dataset[0]
g

`train_mask`, `val_mask` and `test_mask` are bit masks that denote the rows in the `label` and `feat` [Schemes](https://github.com/dmlc/dgl/blob/master/python/dgl/frame.py#L125) which with `DGLBACKEND=pytorch` contain DGL mappings to the [torch.Tensors](https://pytorch.org/docs/stable/tensors.html) making up the training, validation and test datasets respectively.

In [ ]:
print("Node features")
print(g.ndata["feat"].shape)

print("Edge features")
print(g.edata)

### GCN Model Architecture - Diagrams, then Code

The model itself is a PyTorch [torch.nn.Module](https://pytorch.org/docs/stable/generated/torch.nn.Module.html) that uses the [dgl.nn.conv.GraphConv](https://docs.dgl.ai/generated/dgl.nn.pytorch.conv.GraphConv.html) class. 

<center><img src="images/Schematic-diagram-of-a-two-layer-GCN-model-The-dark-green-denotes-target-nodes-that-need_W640.jpg" alt="Diagram of 2-layer GCN from Graph neural networks in node classification: survey and evaluation, Xiao et al., 2022" width="600px" /></center>

<br />

<center>Image credit: <a href="https://www.researchgate.net/publication/355873169_Graph_neural_networks_in_node_classification_survey_and_evaluation">Diagram of 2-layer GCN from Graph neural networks in node classification: survey and evaluation, Xiao et al., 2022</a></center>

<br />

Let's dig into this diagram of our GCN before coding it in DGL.

### Over Smoothing in GNNs: Too Many Layers Means Too Many Hops Sampled

Note that **each layer of the GCN represents a round of message passing where nodes aggregate information from their neighbors.** This is important to know, as if you have too many layers in a GNN, you run into the [oversmoothing problem](https://towardsdatascience.com/over-smoothing-issue-in-graph-neural-network-bddc8fbc2472) where nodes start to look the same as all the other nodes.

<center><img src="images/GNN-oversmoothing-first-layer.webp" width="840px" alt="First layer of GNN message passing, aggregation and summarization results in features of different colors" /></center>
<center>The first layer of GNN message passing, aggregation and summarization results in features represented by different colors.</center>
<center><i>Image credit: <a href="https://towardsdatascience.com/over-smoothing-issue-in-graph-neural-network-bddc8fbc2472">Over-smoothing issue in graph neural network</a> by <a href="https://towardsdatascience.com/over-smoothing-issue-in-graph-neural-network-bddc8fbc2472">Anas Ait Aomar</a></i></center>

<br /><br />

<center><img src="images/GNN-oversmoothing-second-layer.webp" width="1000px" alt="Second layer of GNN message passing, aggregation and summarization results in features with more similar colors" /></center>
<center>The second layer of GNN message passing, aggregation and summarization results in features represented by more similar colors.</center>

<center><i>Image credit: <a href="https://towardsdatascience.com/over-smoothing-issue-in-graph-neural-network-bddc8fbc2472">Over-smoothing issue in graph neural network</a> by <a href="https://towardsdatascience.com/over-smoothing-issue-in-graph-neural-network-bddc8fbc2472">Anas Ait Aomar</a></i></center>

### Relu Activation Function

Note how the GraphConv layers in the GCN architecture diagram above are separated by a Relu layer. Without this layer, the GCN could not learn effectively. Relu is an activation function that enables nonlinearity in neural networks - it lets them model messy data in a way that is much more powerful than a linear model. Relu is defined as `max(0, x)` which means that it maps negative values to 0 and positive values are left alone. Note that there are many derivatives of Relu that attempt to improve its performance.

<center><img src="images/relu.png" width="600px" alt="Relu is max(0, x), making its plot flat when x is less than zero, and evently diagonal in a 1:1 ratio when x is greater than zero." /></center>
<center>The Relu activation function: <code>max(0, x)</code></center>
<center><i>Image Credit: <a href="https://medium.com/@danqing/a-practical-guide-to-relu-b83ca804f1f7">A Practical Guide to ReLU</a> by <a href="https://medium.com/@danqing">Danqing Liu</a></i></center>

> we can stack as many linear classifiers as we want on top of each other, and without nonlinear functions between them, it will just be the same as one linear classifier.
>
> But if we put a nonlinear function between them, such as max, then this is no longer true. Now each linear layer is actually somewhat decoupled from the other ones and can do its own useful work. The max function operates as a simple if statement.
>
_Source: [Nonlinearity and Neural Networks](https://medium.com/unpackai/nonlinearity-and-neural-networks-2ffaaac0e6ff) by [Aravinda 加阳](https://medium.com/@aravinda-gn)_

This video by [deeplizard on Youtube](https://www.youtube.com/@deeplizard) explains Relu and its significance:

In [ ]:
%%HTML
<center><iframe width="800" height="460" src="https://www.youtube.com/embed/6MmGNZsA5nI?si=sglt8BijkpykWdWP&amp;start=10"></iframe></center>

### Coding the Above GCN Diagram

The equivalent DGL code for the GCN diagram above appears below. The graph structure and CORA BoW features are shown as the input, which feeds into one GCN layer, then a Relu activaton function, another GCN layer and finally they are mapped into the labels of our classes, in this case fields of study.

In [ ]:
from dgl.nn import GraphConv


class GCN(nn.Module):
    """2-layer Graph Convolutional Network"""
    
    def __init__(self, in_feats, h_feats, num_classes):
        """Setup two GCN layers of with the input, inner and output dimensions."""
        super(GCN, self).__init__()
        self.conv1 = GraphConv(in_feats, h_feats)
        self.conv2 = GraphConv(h_feats, num_classes)

    def forward(self, g, in_feat):
        """Operate a forward pass of the network"""
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h


# Create the model with given dimensions
model = GCN(g.ndata["feat"].shape[1], 16, dataset.num_classes)

In [ ]:
model

### Training a GCN

Below we define a training function that will iteratively train our GCN using message passing.

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


def metrics(y_true, y_pred):

    return {
        "accuracy": accuracy_score(y_true, y_pred),
        "precision": precision_score(y_true, y_pred, average="micro"),
        "recall": recall_score(y_true, y_pred, average="micro"),
        "f1": f1_score(y_true, y_pred, average="micro"),
    }


def train(g, model):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    best_val_acc = 0
    best_test_acc = 0

    features = g.ndata["feat"]
    labels = g.ndata["label"]
    train_mask = g.ndata["train_mask"]
    val_mask = g.ndata["val_mask"]
    test_mask = g.ndata["test_mask"]
    for e in range(100):
        # Forward
        logits = model(g, features)
        # print(logits.shape)

        # Compute prediction
        pred = logits.argmax(1)

        # Compute loss
        # Note that you should only compute the losses of the nodes in the training set.
        loss = F.cross_entropy(logits[train_mask], labels[train_mask])

        # Compute accuracy on training/validation/test
        train_acc = (pred[train_mask] == labels[train_mask]).float().mean()
        val_acc = (pred[val_mask] == labels[val_mask]).float().mean()
        test_acc = (pred[test_mask] == labels[test_mask]).float().mean()

        train_scores = metrics(labels[train_mask], pred[train_mask])
        val_scores = metrics(labels[val_mask], pred[val_mask])
        test_scores = metrics(labels[test_mask], pred[test_mask])

        # Save the best validation accuracy and the corresponding test accuracy.
        if best_val_acc < val_acc:
            best_val_acc = val_acc
            best_test_acc = test_acc

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if e % 5 == 0:
            print(
                f"In epoch {e}, loss: {loss:.3f}, val acc: {val_acc:.3f} (best {best_val_acc:.3f}), test acc: {test_acc:.3f} (best {best_test_acc:.3f}),",
                f'val precision: {val_scores["precision"]:.3f}, val recall: {val_scores["recall"]:.3f}, val f1: {val_scores["f1"]:.3f}'
            )


model = GCN(g.ndata["feat"].shape[1], 16, dataset.num_classes)
train(g, model)

## Graph Attention Networks (GATs)

Let's try a more sophisticated architecture for node classification.

In [ ]:
from dgl.nn import GATConv


class GAT(nn.Module):
    def __init__(self, in_dim, hidden_dim, out_dim, num_heads):
        super(GAT, self).__init__()
        self.layer1 = GATConv(in_dim, hidden_dim, num_heads=num_heads, activation=F.relu, feat_drop=0.3, attn_drop=0.1)
        self.layer2 = GATConv(hidden_dim * num_heads, out_dim, num_heads=1)

    def forward(self, g, in_feat):
        h = self.layer1(g, in_feat)
        h = h.view(h.shape[0], -1)
        h = self.layer2(g, h)
        return h.squeeze(1)


gatconv = GAT(g.ndata["feat"].shape[1], 10, dataset.num_classes, num_heads=2)
train(g, gatconv)

## GraphDataLoaders

I have written a GraphDataLoader for our dataset in [`graphml_class/citation/dgl.py`](graphml_class/citation/dgl.py).

# Link Prediction with GNNs

In [ ]:
import dgl.data

dataset = dgl.data.CoraGraphDataset()
g = dataset[0]

In [ ]:
import scipy.sparse as sp

# Split edge set for training and testing
u, v = g.edges()

u, v

In [ ]:
eids = np.arange(g.number_of_edges())
eids = np.random.permutation(eids)

test_size = int(len(eids) * 0.1)
train_size = g.number_of_edges() - test_size

test_pos_u, test_pos_v = u[eids[:test_size]], v[eids[:test_size]]
train_pos_u, train_pos_v = u[eids[test_size:]], v[eids[test_size:]]

# Find all negative edges and split them for training and testing
adj = sp.coo_matrix((np.ones(len(u)), (u.numpy(), v.numpy())))
adj_neg = 1 - adj.todense() - np.eye(g.number_of_nodes())
neg_u, neg_v = np.where(adj_neg != 0)

neg_eids = np.random.choice(len(neg_u), g.number_of_edges())
test_neg_u, test_neg_v = neg_u[neg_eids[:test_size]], neg_v[neg_eids[:test_size]]
train_neg_u, train_neg_v = neg_u[neg_eids[test_size:]], neg_v[neg_eids[test_size:]]

In [ ]:
train_g = dgl.remove_edges(g, eids[:test_size])

In [ ]:
from dgl.nn import SAGEConv

# ----------- 2. create model -------------- #
# build a two-layer GraphSAGE model
class GraphSAGE(nn.Module):
    def __init__(self, in_feats, h_feats):
        super(GraphSAGE, self).__init__()
        self.conv1 = SAGEConv(in_feats, h_feats, 'mean')
        self.conv2 = SAGEConv(h_feats, h_feats, 'mean')
        # self.conv3 = SAGEConv(h_feats, h_feats, 'mean')

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        # h = self.conv3(g, h)
        return h

In [ ]:
train_pos_g = dgl.graph((train_pos_u, train_pos_v), num_nodes=g.number_of_nodes())
train_neg_g = dgl.graph((train_neg_u, train_neg_v), num_nodes=g.number_of_nodes())

test_pos_g = dgl.graph((test_pos_u, test_pos_v), num_nodes=g.number_of_nodes())
test_neg_g = dgl.graph((test_neg_u, test_neg_v), num_nodes=g.number_of_nodes())

In [ ]:
import dgl.function as fn

class DotPredictor(nn.Module):
    def forward(self, g, h):
        with g.local_scope():
            g.ndata['h'] = h
            # Compute a new edge feature named 'score' by a dot-product between the
            # source node feature 'h' and destination node feature 'h'.
            g.apply_edges(fn.u_dot_v('h', 'h', 'score'))
            # u_dot_v returns a 1-element vector for each edge so you need to squeeze it.
            return g.edata['score'][:, 0]

In [ ]:
model = GraphSAGE(train_g.ndata['feat'].shape[1], 16)
# You can replace DotPredictor with MLPPredictor.
#pred = MLPPredictor(16)
pred = DotPredictor()

def compute_loss(pos_score, neg_score):
    scores = torch.cat([pos_score, neg_score])
    labels = torch.cat([torch.ones(pos_score.shape[0]), torch.zeros(neg_score.shape[0])])
    return F.binary_cross_entropy_with_logits(scores, labels)

def compute_auc(pos_score, neg_score):
    scores = torch.cat([pos_score, neg_score]).numpy()
    labels = torch.cat(
        [torch.ones(pos_score.shape[0]), torch.zeros(neg_score.shape[0])]).numpy()
    return roc_auc_score(labels, scores)

In [ ]:
import itertools

# ----------- 3. set up loss and optimizer -------------- #
# in this case, loss will in training loop
optimizer = torch.optim.Adam(itertools.chain(model.parameters(), pred.parameters()), lr=0.01)

# list(itertools.chain(model.parameters(), pred.parameters()))

In [ ]:
# ----------- 4. training -------------------------------- #
all_logits = []
for e in range(100):
    # forward
    h = model(train_g, train_g.ndata['feat'])
    pos_score = pred(train_pos_g, h)
    neg_score = pred(train_neg_g, h)
    loss = compute_loss(pos_score, neg_score)

    # backward
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if e % 5 == 0:
        print('In epoch {}, loss: {}'.format(e, loss))

# ----------- 5. check results ------------------------ #
from sklearn.metrics import roc_auc_score
with torch.no_grad():
    pos_score = pred(test_pos_g, h)
    neg_score = pred(test_neg_g, h)
    print('AUC', compute_auc(pos_score, neg_score))